# 准备数据

In [40]:
import random

N = 10

# 1. 随机数据
randomData = list(range(N))
random.shuffle(randomData)

# 2. 倒序数据
reversedData = list(range(0,N,-1))

# 3. 几乎已经排序好的数据
almostSortedData = list(range(N))
almostSortedData[1],almostSortedData[-2] = almostSortedData[-2], almostSortedData[1]

# 4. 已经排序好的数据
sortedData = list(range(N))

# 5. 全部重复的数据
repeatData = [1] * N

# 各种排序算法的代码实现

In [35]:
# 所有排序算法要调用的函数
def swap(data, i, j) -> None:
    data[i], data[j] = data[j], data[i]

## 选择排序

In [31]:
def select_sort(data, reverse=False) -> bool:
    '''选择排序
    参数：
        data：待排序的列表
        reverse: 是否倒序排序，默认False
    返回值：
        bool类型：是否排序成功
    '''
    if not all([str(i).isdigit() for i in data]):
        return False
    
    lenData = len(data)
    
    for i in range(lenData):
        minInd = i
        for j in range(i+1, lenData):
            if data[j] < data[minInd]:
                minInd = j 
        swap(data, i, minInd)
    
    if reverse:
        data = data[::-1]
    return True

In [50]:
import copy

data = copy.copy(randomData)
res = select_sort(data)
res, data, randomData

(True, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [0, 8, 9, 4, 2, 6, 7, 1, 3, 5])

## 插入排序

In [36]:
def insert_sort(data, reverse=False) -> bool:
    '''插入排序
    参数：
        data：待排序的列表
        reverse: 是否倒序排序，默认False
    返回值：
        bool类型：是否排序成功
    '''
    if not all([str(i).isdigit() for i in data]):
        return False
    
    lenData = len(data)
    
    for i in range(1, lenData):
        for j in range(i, 0, -1):
            if data[j] < data[j-1]:
                swap(data, j, j-1)
    
    if reverse:
        data = data[::-1]
    return True

In [51]:
import copy

data = copy.copy(randomData)
res = insert_sort(data)
res, data, randomData

(True, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [0, 8, 9, 4, 2, 6, 7, 1, 3, 5])

## 冒泡排序

In [52]:
def bubble_sort(data, reverse=False) -> bool:
    '''冒泡排序
    参数：
        data：待排序的列表
        reverse: 是否倒序排序，默认False
    返回值：
        bool类型：是否排序成功
    '''
    if not all([str(i).isdigit() for i in data]):
        return False
    
    lenData = len(data)
    
    for i in range(lenData-1):
        for j in range(lenData-1-i):
            if data[j] > data[j+1]:
                swap(data, j, j+1)
    
    if reverse:
        data = data[::-1]
    return True

In [53]:
import copy

data = copy.copy(randomData)
res = bubble_sort(data)
res, data, randomData

(True, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [0, 8, 9, 4, 2, 6, 7, 1, 3, 5])

## 快排
**基本原理**（分治法思想）  
1. 选定Pivot中心轴
2. 遍历尚未处理的数据（乱堆），将小于Pivot的数字放在Pivot左边（小堆），大于Pivot的数字放在Pivot右边（大堆）
3. 分别对左右子序列重复前两步操作

In [56]:
def quick_sort(data, reverse=False) -> bool:
    '''快速排序
    参数：
        data：待排序的列表
        reverse: 是否倒序排序，默认False
    返回值：
        bool类型：是否排序成功
    '''
    if not all([str(i).isdigit() for i in data]):
        return False
    
    lenData = len(data)
    
    quickSort(data, 0, lenData-1)
    
    if reverse:
        data = data[::-1]
    return True


def quickSort(data, l, r):
    '''实现快速排序'''
    if l < r:
        m = get_mid(data, l, r)
        quickSort(data, l, m-1)
        quickSort(data, m+1, r)

        
def get_mid(data, l, r):
    '''将数据分成小堆和大堆，并返回Pivot的索引'''
    pivot = data[l]
    
    while l < r:
        while data[r] > pivot and l < r:
            r -= 1
        else:
            #data[l] = data[r]
            swap(data, l, r)
        while data[l] < pivot and l < r:
            l += 1
        else:
            #data[r] = data[l]
            swap(data, l, r)
    data[l] = pivot
    return l

In [57]:
import copy

data = copy.copy(randomData)
res = quick_sort(data)
res, data, randomData

(True, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [0, 8, 9, 4, 2, 6, 7, 1, 3, 5])

## 归并排序

## 堆排

## 桶排序

# 测试各算法的效率